# Amazon Personalize：将文本用作非结构化项目元数据

您使用 Amazon Personalize 提供的推荐的相关性取决于生成推荐时可用的数据。Amazon Personalize 使用用户的历史交互、项目的属性和用户的元数据来了解哪些项目与每个用户最相关。Amazon Personalize 所需的主要数据是用户-项目交互数据。用户与目录中项目的交互，例如单击产品、阅读文章、观看视频或购买产品，是他们过去找到相关项目的重要信号。添加项目和用户属性（也称为元数据），可以增强推荐的相关性；尤其是对于与用户找到的相关项目相似的新项目。但是，结构化元数据（例如项目的类别、样式或流派）可能并不总是现成可用，或者无法提供叙事描述中的所有信息。现在，Amazon Personalize 允许您将非结构化元数据（例如产品描述、视频转录内容或文章文本）添加到其他项目属性中。Amazon Personalize 可以托管、管理、自动使用自然语言处理（NLP）模型来处理文本，并使用它来提高 Amazon Personalize 解决方案的性能。

此笔记本将演示如何将产品描述形式的文本作为非结构化项目元数据包含在内，以提高推荐的相关性。

来自 Amazon Prime Pantry 类别的 Amazon Reviews 数据用于交互和项目数据集。

考虑在项目数据集中包含文本时，请记住以下最佳做法。
- 经过编辑验证的文本对于每个项目都是简洁、相关的并且信息丰富，其中最相关的细节在文本前面提到，优先于用户生成的可能不太相关或一致的内容
- 稀疏填充的文本列将减少在项目数据集中包含文本的积极影响
- 在将标记文本和多余的空白格式文本添加到项目数据集之前，先清除所有标记文本和多余的空白格式文本
- 英语是此文本字段当前支持的唯一语言
- 目前仅对 User-Personalization 和 Personalized-Ranking 配方考虑文本字段

将创建两个数据集组，其中将包含带项目描述和不带项目描述的数据，以便我们可以训练单独的模型并比较其离线和在线结果。

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## 加载和检查数据集

我们将首先加载 Prime Pantry 评论数据集。您需要填写表单才能访问数据文件：

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

引文：
> 使用距离标记评论和细粒度方面来证明推荐的合理性  
> Jianmo Ni、Jiacheng Li、Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

mkdir: cannot create directory ‘raw_data’: File exists
--2021-07-13 22:06:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  12.3MB/s    in 4.8s    

2021-07-13 22:06:57 (9.01 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-07-13 22:06:57--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pa

### 加载并检查评论数据

我们将首先加载 Prime Pantry 产品的评论数据集，并运行一些命令来查看需要处理的内容。

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

我们可以从此输出中学到什么？ 有超过 47.1 万条评论和 12 列数据。`asin` 列是我们的唯一项目标识符，`reviewerID` 是我们的唯一用户标识符，`unixReviewTime` 是我们的评论时间戳，`overall` 表示按照 1-5 的等级对评论积极性的评级。我们将使用此文件作为 Personalize 交互数据集的基础。 

### 构建和保存交互数据集

让我们通过减少我们想要包含的行来开始构建我们的交互数据集。第一步是只分离积极的评论。为此，我们将假设总体评级为 4 或更高的任何评论都是积极的评论。评级为 3 或以下的任何评论都是一般或负面评论。

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

我们的正面评论已减少到 38.7 万条。仍然有足够的数据来训练 Personalize 模型。

接下来，让我们将数据集的范围缩小到所需的列，并添加一个 `EVENT_TYPE` 列来指示要捕获的事件类型。如果您选择这样做，现在添加一个 `EVENT_TYPE` 列将使以后探索测试实时事件更简单（因为 `eventType` 是 [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html) API 的必填字段）。

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


我们应该做的最后一项检查是对 `unixReviewTime` 列值进行健全性检查。由于 Personalize 基于每个交互的日期和时间构建序列模型，因此，以预期的格式表示每个交互的时间戳非常重要，以便能够正确解释。

让我们为 `unixReviewTime` 列选择一个值并将该值解析为人类可读的日期，以便我们可以验证它是否合理。

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


时间戳值看起来很好。让我们获取数据集的一些最终摘要信息。

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


我们有 20.2 万不同评论者/用户针对 1 万个不同产品的 38.7 条评论。这是我们交互数据集的基础。

但是，在我们将这些信息用作交互数据集之前，我们需要重命名列以匹配 Personalize 所要求的列。

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


最后，让我们将正面评论数据框另存为 CSV。我们稍后会在此笔记本中将此 CSV 上载到 Personalize。

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### 加载和检查项目元数据

现在，我们已经建立了交互数据集，让我们看一下项目数据集。在这里，我们将找到我们将包含在模型中的非结构化文本值。

与评论数据集一样，Prime Pantry 项目元数据文件也以 JSON 表示。由于此文件的嵌套性质，这将在按照我们需要的方式格式化数据方面带来一些挑战。

让我们首先将元数据文件加载到数据框中并查看数据。

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Infants' Motrin Concentrated Drops, Fever Redu...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


那么，我们可以从这些信息中了解到什么？ 首先，元数据文件中包含了超过 1 万种产品。大多数列对我们使用 Personalize 没有什么价值，因为它们与功能（图像 URL、`details`、`also_viewed`、`also_buy` 等）无关，或者大部分是空白/稀疏的列（`category`、`fit`、`tech1` 等）。`asin` 列是每个项目的唯一标识符（尽管看起来有一个重复），`brand` 和 `price` 看起来可能很有用。`description` 列是我们将用于非结构化文本的内容。

然而，我们必须对要在项目数据集中使用的字段进行一些清理和重新格式化。例如，由于这些值的表示方式和从原始 JSON 文件中解析的方式，`price` 字段是格式化的货币值（字符串）而不是数字，`description` 字段被加载为字符串数组。最后，`description` 值还包含需要剥离的 HTML 标记。

让我们首先创建一个数据框，其中仅包含项目数据集所需的列。

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


接下来，让我们根据 `asin` 列值删除重复的行。根据上面的 `describe()` 输出，应该只有一个副本。

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

接下来，让我们专注于重新格式化和清理 `description` 列值。正如您在上面看到的，`description` 当前表示为字符串数组（因为这就是它在 JSON 文件中的表示方式）。我们需要将该数组扁平化为单个字符串，并从每个片段中剥离所有 HTML 标记。

我们将首先创建两个实用工具函数用于清理 `description`（稍后创建我们想要显示推荐产品的标题时原始数据集中的 `title` 列）。

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


接下来，让我们看一下 `price` 列并将该列类型从字符串更改为浮点数。

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$20.42       1
$32.32       1
$1.52        1
$27.89       1
$39.10       1
Name: price, Length: 1482, dtype: int64

以下单元格会将空/非数字价格转换为 `np.nan`，所有其他单元格将删除 `$` 货币符号。这将允许我们将类型强制为浮点数。

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
39.10      1
1.84       1
22.95      1
12.17      1
11.09      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

接下来，我们将重命名列以匹配 Personalize 要求的名称和大写名称格式。

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


我们将创建两个项目 CSV。一个会有描述栏，另一个没有。我们将分别使用两个 CSV 来训练具有相同配方的单独模型，以便我们可以比较离线指标并对建议进行一些在线检查。

In [22]:
items_with_desc_filename = "items-with-desc.csv"
items_df.to_csv(items_with_desc_filename, index=False, float_format='%.2f')

删除了包含描述列的另一个项目 CSV。

In [23]:
items_without_desc_df = items_df[['ITEM_ID', 'BRAND', 'PRICE']]
items_without_desc_df.head()

,ITEM_ID,BRAND,PRICE
1,B00005BPJO,Milk Duds,5.0
2,B0000DIF38,Goya,NaN
3,B0000DIWNI,Saran,NaN
4,B0000DIWNZ,Saran,NaN
5,B0000GH6UG,Ibarra,NaN


In [24]:
items_without_desc_filename = "items-without-desc.csv"
items_without_desc_df.to_csv(items_without_desc_filename, index=False, float_format='%.2f')

## 创建数据集组并上载数据集

有了我们需要构建的数据集，现在该使用数据集导入作业将它们上载到 Personalize 了。在上载 CSV 之前，我们需要创建数据集组来保存两种数据集方法（不带描述和带描述），为数据集创建架构，并创建数据集。

我们将首先创建需要与 Personalize 交互的 SDK 客户端。

In [25]:
personalize = boto3.client('personalize')

### 创建数据集组

让我们创建两个数据集组。

In [26]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-without-desc"
)

dataset_group_without_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc",
  "ResponseMetadata": {
    "RequestId": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-with-desc"
)

dataset_group_with_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc",
  "ResponseMetadata": {
    "RequestId": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


由于数据集组可能需要几秒钟才能完全创建，所以，让我们等到数据集组都处于 ACTIVE 状态。

In [28]:
in_progress_dataset_group_arns = [ dataset_group_without_desc_arn, dataset_group_with_desc_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc


### 创建交互数据集架构和数据集

由于两个数据集组的交互数据集是相同的，因此我们将为交互数据集类型创建一个架构，并在两个数据集组之间共享此架构。这是可能的，因为架构对于 AWS 账户是全局的，而不是特定于数据集组。

In [29]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-interactions",
  "ResponseMetadata": {
    "RequestId": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:40 GMT",
      "x-amzn-requestid": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在两个数据集组中创建一个交互数据集，指定刚才创建的架构。

In [30]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "614257cc-6344-408e-8e15-89d4f81ae927",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "614257cc-6344-408e-8e15-89d4f81ae927",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### S3 中的阶段交互 CSV

在将之前创建的交互 CSV 上载到刚刚创建的 Personalize 数据集中之前，我们需要将此 CSV 暂存到 S3 存储桶中。

让我们创建一个 S3 存储桶并将交互 CSV 文件复制到存储桶中。

In [32]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [33]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "amazon-pantry-personalize-text"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

224124347618-us-east-1-amazon-pantry-personalize-text


#### 将交互 CSV 上载到 S3

In [34]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### 创建 S3 存储桶策略和 IAM 角色

在我们向 Personalize 提交数据集导入作业之前，我们必须创建一个存储桶策略和 IAM 角色，以便 Personalize 可以访问我们的存储桶。

In [35]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'SBN10P9R7H7ST5RK',
  'HostId': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
   'x-amz-request-id': 'SBN10P9R7H7ST5RK',
   'date': 'Tue, 13 Jul 2021 22:10:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [36]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantry"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::224124347618:role/PersonalizeRoleAmazonPantry


### 为每个数据集组导入交互数据集

现在，我们已准备好将 S3 存储桶中的分段交互 CSV 导入我们在每个数据集组中创建的 Personalize 数据集。我们将提交两个导入作业并等待它们完成。

In [37]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-ints-import",
    datasetArn = interactions_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:08 GMT",
      "x-amzn-requestid": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [38]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-ints-import",
    datasetArn = interactions_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "eae39716-264b-48e8-bfb4-93f569c7a904",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:09 GMT",
      "x-amzn-requestid": "eae39716-264b-48e8-bfb4-93f569c7a904",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待交互数据集导入作业完成

以下单元格将等待两个导入作业完成。

In [39]:
%%time

in_progress_import_arns = [ dataset_import_job_without_ints_arn, dataset_import_job_with_ints_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import
CPU times: user 42.3 ms, sys: 6.1 ms, total: 48.4 ms
Wall time: 5min


### 创建项目数据集架构和数据集

接下来，我们将对项目数据集重复此过程。不过，这一次我们需要创建两个架构，因为一个项目数据集包含描述列，另一个不包含描述列。我们将从不包含描述的架构开始。

In [40]:
item_without_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-without-desc-schema",
    schema = json.dumps(item_without_desc_schema)
)

item_without_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-without-desc-schema",
  "ResponseMetadata": {
    "RequestId": "11a18b05-189b-4485-ba8b-58083e784321",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "11a18b05-189b-4485-ba8b-58083e784321",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将创建一个包含描述的架构。请务必记下 `DESCRIPTION` 字段上的 `"textual": True` 属性。这是您区分非结构化文本字段与分类字段和字符串字段的方式。如果没有此属性，Personalize 将不会应用自然语言处理技术从该文本中提取特征。

In [41]:
item_with_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-with-desc-schema",
    schema = json.dumps(item_with_desc_schema)
)

item_with_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-with-desc-schema",
  "ResponseMetadata": {
    "RequestId": "05c60e80-1d7c-45f1-a881-d08af62a8432",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "05c60e80-1d7c-45f1-a881-d08af62a8432",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在每个数据集组中创建 Personalize 数据集，并特别注意为每个数据集指定相应的架构 ARN。

In [42]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = item_without_desc_schema_arn
)

items_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:13 GMT",
      "x-amzn-requestid": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [43]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = item_with_desc_schema_arn
)

items_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:16 GMT",
      "x-amzn-requestid": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### S3 中的阶段项目 CSV

接下来，我们将两个项目 CSV 文件复制到上面创建的同一个 S3 存储桶中。

In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_without_desc_filename).upload_file(items_without_desc_filename)

In [45]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_with_desc_filename).upload_file(items_with_desc_filename)

### 为每个数据集组导入项目数据集

由于已设置 S3 存储桶策略和 IAM 角色，我们只需提交两个数据集导入作业即可导入项目 CSV。

In [46]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-items-import",
    datasetArn = items_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_without_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:37 GMT",
      "x-amzn-requestid": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [47]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-items-import",
    datasetArn = items_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_with_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "cf95206f-1527-4247-a618-6c8c832fa05f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:38 GMT",
      "x-amzn-requestid": "cf95206f-1527-4247-a618-6c8c832fa05f",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待项目导入作业完成

以下逻辑将等到两个项目数据集完全导入每个数据集组。

In [48]:
%%time

in_progress_import_arns = [ dataset_import_job_without_items_arn, dataset_import_job_with_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import
CPU times: user 57.6 ms, sys: 5.06 ms, total: 62.7 ms
Wall time: 7min


## 创建解决方案和解决方案版本

将交互和项目数据集导入每个数据集组后，我们接下来将使用每个数据集组中的数据的 user-personalization 配方创建解决方案和解决方案版本。

首先，让我们列出可用的 Personalize 配方。

In [49]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

我们将为此笔记本使用 user-personalization 配方，因为它是使用项目元数据的配方之一。此配方支持规范的个性化用例，在这样的用例中给定用户，您希望 Personalize 推荐该用户感兴趣的项目。 

In [50]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

首先，我们将在数据集组中创建不包含项目描述的解决方案和解决方案版本。

In [51]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-without-desc-userpersonalization",
    datasetGroupArn = dataset_group_without_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_without_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [52]:
print(user_personalization_without_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization


In [53]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_without_desc_solution_arn
)

In [54]:
user_personalization_without_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "ResponseMetadata": {
    "RequestId": "018b3fcb-10d5-4290-a17a-3970723abacd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:14 GMT",
      "x-amzn-requestid": "018b3fcb-10d5-4290-a17a-3970723abacd",
      "content-length": "132",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在包含项目描述的数据集组中创建解决方案和解决方案版本。

In [55]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-with-desc-userpersonalization",
    datasetGroupArn = dataset_group_with_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_with_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [56]:
print(user_personalization_with_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization


In [57]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_with_desc_solution_arn
)

In [58]:
user_personalization_with_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "ResponseMetadata": {
    "RequestId": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:29 GMT",
      "x-amzn-requestid": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
      "content-length": "129",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待解决方案版本变为活跃状态

最后，我们将等待创建解决方案版本完成。在此步骤中，Personalize 基于数据集和所选配方训练机器学习模型。Personalize 还会将交互数据集拆分为训练部分和评估部分，这样它就可以使用保留的数据根据训练的模型评估推荐的质量。

您会注意到，与数据集组中不包含描述数据的解决方案版本相比，数据集中包含描述数据的解决方案版本需要更长的训练时间。

In [59]:
%%time

in_progress_solution_versions = [
    user_personalization_without_solution_version_arn,
    user_personalization_with_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

一般来说，添加基于文本的非结构化元数据会增加训练时间。在我们的案例中，您可以在上面看到，与基于不包含生产描述的数据集进行训练的解决方案版本相比，基于包含产品描述的数据集进行训练的解决方案版本所用的时间大约多 15 分钟。这种差异会因数据集的组成和文本值而异。

让我们检查每个解决方案版本的训练小时数并进行比较。

In [60]:
response = personalize.describe_solution_version(solutionVersionArn = user_personalization_without_solution_version_arn)
training_hours_without_desc = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = user_personalization_with_solution_version_arn)
training_hours_with_desc = response['solutionVersion']['trainingHours']
training_diff = (training_hours_with_desc - training_hours_without_desc) / training_hours_without_desc

print(f"Training hours without description: {training_hours_without_desc}")
print(f"Training hours with description: {training_hours_with_desc}")

print("Difference of {:.2%}".format(training_diff))

Training hours without description: 4.199
Training hours with description: 5.346
Difference of 27.32%


用于成本计算的训练小时数比使用描述列的训练小时数多约 27%。

挂钟时间和训练小时数将根据数据集大小而有所不同，但此信息可以帮助您在考虑向数据集添加非结构化文本时评估权衡。

### 检查离线指标

现在，解决方案版本已经构建完成，让我们检查并比较每个解决方案版本的离线指标，看看包含非结构化文本如何影响这些指标。

In [61]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_without_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "metrics": {
    "coverage": 0.0914,
    "mean_reciprocal_rank_at_25": 0.0268,
    "normalized_discounted_cumulative_gain_at_10": 0.0376,
    "normalized_discounted_cumulative_gain_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_5": 0.0309,
    "precision_at_10": 0.0058,
    "precision_at_25": 0.0037,
    "precision_at_5": 0.0076
  },
  "ResponseMetadata": {
    "RequestId": "8c61339a-f929-47e0-81f0-a9660ebd589f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:07 GMT",
      "x-amzn-requestid": "8c61339a-f929-47e0-81f0-a9660ebd589f",
      "content-length": "430",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


让我们将这些指标保存在字典中，以便我们可以更轻松地比较两个解决方案版本之间的指标。

In [62]:
metrics = {
    'Coverage': [ metrics_response['metrics']['coverage'] ],
    'MRR-25': [ metrics_response['metrics']['mean_reciprocal_rank_at_25'] ],
    'NDCG-5': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'] ],
    'NDCG-10': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'] ],
    'NDCG-25': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'] ],    
    'Precision-5': [ metrics_response['metrics']['precision_at_5'] ],
    'Precision-10': [ metrics_response['metrics']['precision_at_10'] ],
    'Precision-25': [ metrics_response['metrics']['precision_at_25'] ],    
}

接下来，获取包含描述列的解决方案版本的离线指标并将它们也保存下来。

In [63]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_with_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "metrics": {
    "coverage": 0.1323,
    "mean_reciprocal_rank_at_25": 0.0367,
    "normalized_discounted_cumulative_gain_at_10": 0.049,
    "normalized_discounted_cumulative_gain_at_25": 0.0591,
    "normalized_discounted_cumulative_gain_at_5": 0.0425,
    "precision_at_10": 0.0071,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b54df693-4378-4194-96c7-cec3a9d934cf",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:14 GMT",
      "x-amzn-requestid": "b54df693-4378-4194-96c7-cec3a9d934cf",
      "content-length": "426",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
metrics['Coverage'].append(metrics_response['metrics']['coverage'])
metrics['MRR-25'].append(metrics_response['metrics']['mean_reciprocal_rank_at_25'])
metrics['NDCG-5'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'])
metrics['NDCG-10'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'])
metrics['NDCG-25'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'])
metrics['Precision-5'].append(metrics_response['metrics']['precision_at_5'])
metrics['Precision-10'].append(metrics_response['metrics']['precision_at_10'])
metrics['Precision-25'].append(metrics_response['metrics']['precision_at_25'])

计算有文本和无文本的每个指标的百分比变化并显示结果。

In [65]:
for key in metrics:
    metrics[key].append("{:.2%}".format((metrics[key][1] - metrics[key][0])/metrics[key][0]))

metrics_df = pd.DataFrame.from_dict(metrics,orient='index',columns=['Without Text', 'With Text', '% Change'])
metrics_df

,Without Text,With Text,% Change
Coverage,0.0914,0.1323,44.75%
MRR-25,0.0268,0.0367,36.94%
NDCG-5,0.0309,0.0425,37.54%
NDCG-10,0.0376,0.0490,30.32%
NDCG-25,0.0464,0.0591,27.37%
Precision-5,0.0076,0.0104,36.84%
Precision-10,0.0058,0.0071,22.41%
Precision-25,0.0037,0.0045,21.62%


这些指标清楚地表明，包含项目描述的解决方案版本中的推荐在所有方面都明显更好。对于更稀疏的交互，与每个项目和/或用户的交互次数已经更高的数据集相比，添加文本会使用户和项目的交互次数更少的数据集受益更多。

## 清理

可以从 AWS 控制台的 Personalize 服务页面删除由此笔记本创建的 Personalize 资源。

或者，也可以在本地运行以下脚本以删除每个数据集组中的所有资源。

https://gist.github.com/james-jory/62ddddf2f9180b77dd2a42e645b9d3b0

此外，可以分别从 IAM 和 S3 服务页面中删除 IAM 角色和 S3 存储桶。